## Imports

In [ ]:
import requests
import json
import zipfile
import arcpy

## Retrieve the data

In [ ]:
# a function to search MN_Geospatial Commons for specific datasets
def downloader (search_query, result_num, resource_num):
    ##############################
    # the URL is the MNGC API location + search terms you want
    big_url = 'https://gisdata.mn.gov/api/3/action/package_search?q=' + search_query
    
    # Sends a request to the API for the set URL
    # API returns response object
    response = requests.get(big_url, verify = False)
    
    # response object need to be loaded as a JSON
    json_response = json.loads(response.content)
    
    # this digs into the first layer of the JSON to a list of results
    result_options = json_response['result']['results']
    
    #select the result 
    chosen_result = result_options[result_num]
    
    #dig further to resources and select resource number
    resources_under_result= chosen_result['resources'][resource_num]
    
    # find the URL for that resource for retreival
    chosen_resource = resources_under_result['url']
    print(chosen_resource)

    # send a request to the resource URL and get response object
    URL_request = requests.get(chosen_resource)
    
    #save this response object to a zipfile (because response is a ZIP)
    with open('filename.zip', 'wb') as f:
        f.write(URL_request.content)  
        f.close()
        
    #extract the zipfile contents     
    with zipfile.ZipFile("filename.zip","r") as zip_ref:
        zip_ref.extractall('C:\\Users\Cole\Documents\GitHub\GIS5572\SemProj\Raw Data')
    
    #confirm completion
    print('Download and extraction complete. Check notebook folder')
##############################

#execute function for DEM and RoadCenterline datasets
downloader('us-mn-state-metrogis-trans-road-centerlines-gac',7,1)
downloader('dataset/elev-dtm-30m-condpr-a',1,1)

## Create Feature Dataset to work in

In [38]:
#find the spatial reference from
spatial_ref = arcpy.Describe('C:\\Users\Cole\Documents\GitHub\GIS5572\SemProj\Raw Data\RoadCenterline.shp').spatialReference
arcpy.CreateFeatureDataset_management(r'StreetsProject.gdb',
'Networks2', spatial_ref)

<Result 'StreetsProject.gdb\\Networks2'>

## Bring the data into the GDB/Dataset

In [39]:
#can skip is subsequent test
import arcpy
arcpy.env.workspace = 'C:\\Users\Cole\Documents\GitHub\GIS5572\SemProj'
arcpy.FeatureClassToGeodatabase_conversion('Raw Data\RoadCenterline.shp',
                                           'Notebooks\StreetsProject.gdb\\Networks2')
arcpy.CopyRaster_management('Raw Data\digital_terrain_model.gdb\DTM30CONDPR_A',
                                           'Notebooks\StreetsProject.gdb\\DEM')

<Result 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb\\DEM'>

## Calculate elevations for RoadCenterlines from DEM

In [41]:
import arcpy
arcpy.env.workspace = 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb'

roads = "Networks2\\RoadCenterline"
arcpy.AddSurfaceInformation_3d(roads, "DEM", "Z_MAX;Z_MIN", "LINEAR")

<Result 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb\\Networks2\\RoadCenterline'>

## Convert RoadCenterlines units from M to FT

In [43]:
#need to divide by 3.28 to from m to ft. ft is what the roads are in
import arcpy
arcpy.env.workspace = 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb\\Networks2'
# need to convert the lenghths to feet like the elevation    
arcpy.CalculateField_management("RoadCenterline", "LENG_FT", "!Shape_Length!*3.28", "PYTHON3", field_type = 'DOUBLE')

<Result 'RoadCenterline'>

## Find the slope

In [44]:
#now we can find slope w/ units correct
arcpy.env.workspace = 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb\\Networks2'
arcpy.CalculateField_management("RoadCenterline", "Slope", 
                                '(!Z_MAX!-!Z_MIN!)/!LENG_FT!', "PYTHON3", field_type = 'DOUBLE')

<Result 'RoadCenterline'>

## Find the energy cost on the roadlines layer

In [45]:
# e-score: absolute and linear for now - length * slope score
    # might be better if curved: 30' hill takes >2x energy vs. 15' hill NOT TRUE
arcpy.CalculateField_management("RoadCenterline", "E_Score", 
                                '!Slope!*!LENG_FT!', "PYTHON3", field_type = 'DOUBLE')

<Result 'RoadCenterline'>

## Create Network Dataset

In [69]:
arcpy.env.workspace = 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb'
#arcpy.CopyFeatures_management("Networks2\\RoadCenterline", "Networks2\\AllRoads")
arcpy.na.CreateNetworkDataset(r"Networks2", 
                              "All_ND", "RoadCenterline", 
                              "ELEVATION_FIELDS")

arcpy.na.BuildNetwork('Networks2\All_ND')

<Result 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb\\Networks2\\All_ND'>

## Create ND_Layer 

In [1]:
arcpy.env.workspace = 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb'

#inputs/outputs
NDS = r'Networks2/All_ND'
ND_layer = 'Working'
#input_stops = 'Networks2\\SourceModel'
#output_routes = r'C:\\Users\Cole\Documents\GitHub\GIS5572\SemProj\Notebooks\StreetsProject.gdb\Result'
arcpy.nax.MakeNetworkDatasetLayer(NDS, ND_layer)
#current model

#create a newtork dataset layer from the NDS for faster processing
# this ND layer is only temprorary, not sved to hard memeory


## Create TravelMode

In [10]:
# actually have to make from scratch or do in GUI 1 block earlier
arcpy.nax.TravelMode(None,{
 #   (attribute name, parameter : parameter value),
  #  (type : WALK), (useHierarchy: NO_HEIRARCHY)})
nd_travel_modes = arcpy.nax.GetTravelModes(NDS)
travel_mode = nd_travel_modes["???????"]

{'New Travel Mode': <Network Travel Mode object object at 0x000001D54A8D29D0>}


## Create Inputs Layer

In [ ]:
#build the start and end point layer, sourceModel
import arcpy
arcpy.env.workspace = 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb'
spatial_ref = arcpy.Describe("Networks2\\RoadCenterline").spatialReference
#create a new feature class
arcpy.CreateFeatureclass_management("Networks2","SourceModel", "POINT",
                                   spatial_reference = spatial_ref)

## Add Facilities/Stops to Inputs Layer

In [12]:
# AFTER travel modes because so don't have to redo every time if executing lineraly 

##########################################
start = arcpy.Point(484436.82, 4976072.95)
end = arcpy.Point(485814.56, 4974921.37)
###################-change-##############
# add the start and end points to the the sourcemodel
arcpy.env.workspace = 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb\\Networks2'
feature_class_source = "SourceModel"
cursor = arcpy.da.InsertCursor(feature_class_source,"SHAPE@XY")
cursor.insertRow([start])
cursor.insertRow([end])
del cursor

## Initialize Route Solver

In [7]:
## Instantiate a Route solver object
route = arcpy.nax.Route(ND_layer)

ValueError: Input network dataset does not have at least one travel mode.

## Set Route Solver properties

In [ ]:
#set network properties
route.timeUnits = arcpy.nax.TimeUnits.Minutes

route.accumulateAttributeNames = 

route.directionsDistanceUnits = 

route.distanceUnits =

route.findBestSequence =

#route.networkDataSource =

route.preserveFirstStop =

route.preserveLastStop =

route.returnDirections =

route.returnRouteEdges = True

route.returnRouteJunctions = 

route.returnRouteTurns = 

route.returnToStart = 

route.searchTolerance = 

route.searchToleranceUnits = 

route.travelMode = 

## Load Inputs Layer

In [ ]:
route.load(arcpy.nax.RouteInputDataType.Stops, SourceModel)

## Solve the Route Solver

In [ ]:
#solve
result = route.solve()

#error checker
if result.solveSucceeded:
    result.export(arcpy.nax.RouteOutputDataType.Routes, output_routes)
else:
    print("Solved failed")
    print(result.solverMessages(arcpy.nax.MessageSeverity.All)) 

## Depreciated Model/OLD

In [ ]:
#depreciated


#selected manually in ArcPro: GrandAveTest roads set

arcpy.CopyFeatures_management("RoadCenterline", "Networks2\AllRoads")
arcpy.na.CreateNetworkDataset(r"Networks", 
                              "All_ND", "AllRoads", 
                              "ELEVATION_FIELDS")

arcpy.na.BuildNetwork('Networks\Test_ND')
arcpy.na.BuildNetwork('Networks\All_ND')

#depreciated*****

#NAX module
## Source Settings
    ## Vertical connect.
## Travel Attributes
    ## Travel Modes: create one for walking, one for wheelchair
    ## Costs: Energy_Cost = E_score (assign at different quadratic?)
        ## wheelchair energy cost increases faster,
        ## but has a lower slope limit than person (.12% slope vs .3% slope),
        # and slightly higher power limit (say, 450 W vs 400 W)
    ## Restrictions: avoid energy cost, high
## Directions
    ## Support Directions:checked
    ## Field mapping: full name to ST_name
    
arcpy.na.BuildNetwork('Networks\Test_ND')
arcpy.na.BuildNetwork('Networks\All_ND')

#depreciated**********

def routelayers(network, output, mode):
    arcpy.env.workspace = 'C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Notebooks\\StreetsProject.gdb\\Networks'
   
    #create a route analysis layer
    result_object = arcpy.na.MakeRouteAnalysisLayer(network, output, mode,"PRESERVE_BOTH")
   
    #grab the route layer object from the result object layer
    layer_object = result_object.getOutput(0)
    
    # add locations from SourceModel to the route layer object as Stops
    arcpy.na.AddLocations(layer_object, "Stops", "SourceModel")
    
    
    #some network modify commands take pl
    
    
    arcpy.na.Solve(layer_object)
    layer_object.saveACopy("C:\\Users\\Cole\\Documents\\GitHub\\GIS5572\\SemProj\\Output\\"+output)
    
routelayers("All_ND", "Walking", "Walking")
routelayers("All_ND", "Wheelchair", "Wheelchair")